<a href="https://colab.research.google.com/github/Jeorr/ML/blob/main/Lab2PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, Normalize
from pathlib import Path
from torchvision.transforms import ToPILImage
import pandas as pd
import zipfile
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Розархівування zip файлу, якщо він існує
zip_path = '/content/drive/MyDrive/ML/cats_and_dogs.zip'
extract_to_path = '/content/Lab2'
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Файли успішно розархівовано в {extract_to_path}")
else:
    print(f"Файл {zip_path} не знайдено.")

# Підготовка пристрою
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовується пристрій: {device}")

# Параметри
batch_size = 32
num_epochs = 10
learning_rate = 0.001
image_size = (150, 150)

# Шляхи
train_dir = '/content/Lab2/cats_and_dogs/train'
validation_dir = '/content/Lab2/cats_and_dogs/validation'
test_dir = '/content/Lab2/cats_and_dogs/test'

# Пройдемося по всіх файлах у папці
for img_path in Path(test_dir).iterdir():
    if img_path.is_file() and img_path.suffix in [".jpg", ".png", ".jpeg"]:  # Фільтруємо лише зображення
        # Завантажуємо зображення
        img = read_image(str(img_path))
        img = test_transform(to_pil(img)).unsqueeze(0).to(device)

# Трансформації зображень
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Завантаження даних
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
validation_dataset = datasets.ImageFolder(validation_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

# Завантаження попередньо натренованої моделі
base_model = models.efficientnet_b0(pretrained=True)

# Заморожуємо верхні шари
for param in base_model.features.parameters():
    param.requires_grad = False

# Налаштування голови моделі
num_features = base_model.classifier[1].in_features
base_model.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 1),
    nn.Sigmoid()
)
base_model = base_model.to(device)

# Функція втрат та оптимізатор
criterion = nn.BCELoss()
optimizer = optim.Adam(base_model.parameters(), lr=learning_rate)

# Тренування
for epoch in range(num_epochs):
    base_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = base_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    # Валідація
    base_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = base_model(inputs)
            preds = (outputs > 0.5).float()
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    val_accuracy = correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy:.4f}")

# Передбачення на валідаційному наборі
base_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = base_model(inputs)
        preds = (outputs > 0.5).float()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Обчислення метрик
mae = mean_absolute_error(all_labels, all_preds)
mse = mean_squared_error(all_labels, all_preds)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

to_pil = ToPILImage()

# Трансформації для зображень тестового набору
test_transform = Compose([
    Resize(image_size),
    transforms.ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_results = []

# Пройдемося по всіх файлах у папці
for img_path in Path(test_dir).iterdir():
    if img_path.is_file() and img_path.suffix in [".jpg", ".png", ".jpeg"]:  # Фільтруємо лише зображення
        # Завантажуємо зображення
        img = read_image(str(img_path))
        img = test_transform(to_pil(img)).unsqueeze(0).to(device)

        # Виконуємо передбачення
        with torch.no_grad():
            output = base_model(img)
            predicted_class = "Dog" if output.item() > 0.5 else "Cat"

        # Додаємо результат
        test_results.append({"FileName": img_path.name, "PredictedClass": predicted_class})

# Створюємо DataFrame для результатів
test_results_df = pd.DataFrame(test_results)

# Виводимо результати в консоль
print(test_results_df)


Файли успішно розархівовано в /content/Lab2
Використовується пристрій: cpu


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 22.3780, Train Accuracy: 0.8435
Validation Accuracy: 0.9030
Epoch [2/10], Loss: 14.5955, Train Accuracy: 0.9030
Validation Accuracy: 0.9130
Epoch [3/10], Loss: 13.5741, Train Accuracy: 0.9160
Validation Accuracy: 0.9210
Epoch [4/10], Loss: 10.7171, Train Accuracy: 0.9315
Validation Accuracy: 0.9150
Epoch [5/10], Loss: 11.6699, Train Accuracy: 0.9250
Validation Accuracy: 0.9040
Epoch [6/10], Loss: 9.5191, Train Accuracy: 0.9430
Validation Accuracy: 0.9210
Epoch [7/10], Loss: 9.5259, Train Accuracy: 0.9395
Validation Accuracy: 0.9230
Epoch [8/10], Loss: 8.5353, Train Accuracy: 0.9385
Validation Accuracy: 0.9240
Epoch [9/10], Loss: 8.0570, Train Accuracy: 0.9515
Validation Accuracy: 0.9140
Epoch [10/10], Loss: 7.8776, Train Accuracy: 0.9520
Validation Accuracy: 0.9150
Mean Absolute Error (MAE): 0.08500000089406967
Mean Squared Error (MSE): 0.08500000089406967
   FileName PredictedClass
0     4.jpg            Dog
1     2.jpg            Dog
2     1.jpg            Dog
3  